<a href="https://colab.research.google.com/github/adyasha95/feature-tracking-opencv-demos/blob/main/checkerboard_flow_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
Lucas–Kanade optical flow on a synthetic moving checkerboard.
Generates a short sequence, detects corners on the first frame,
and tracks them across frames.

Usage:
  python checkerboard_flow.py
"""

import numpy as np
import matplotlib.pyplot as plt
import cv2

def make_checkerboard(H=480, W=640, s=24, shift_xy=(0, 0)):
    """Create a grayscale checkerboard, optionally shifted by (dx, dy)."""
    img = np.zeros((H, W), dtype=np.uint8)
    dx, dy = shift_xy
    for i in range(0, H, 2 * s):
        for j in range(0, W, 2 * s):
            img[i + dy : i + s + dy, j + dx : j + s + dx] = 255
            img[i + s + dy : i + 2 * s + dy, j + s + dx : j + 2 * s + dx] = 255
    return img

def main():
    # Build a sequence: translation of +2 px (x) and +1 px (y) per frame
    frames = [make_checkerboard(shift_xy=(k * 2, k * 1)) for k in range(20)]

    # Detect Shi–Tomasi corners on first frame
    p0 = cv2.goodFeaturesToTrack(frames[0], maxCorners=150, qualityLevel=0.01, minDistance=8, blockSize=7)
    if p0 is None:
        raise SystemExit("No corners found on the synthetic image. Try changing sizes/params.")
    p0 = np.float32(p0)

    # Lucas–Kanade parameters
    lk_params = dict(
        winSize=(21, 21),
        maxLevel=3,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01),
    )

    mask = np.zeros((frames[0].shape[0], frames[0].shape[1], 3), dtype=np.uint8)  # to accumulate tracks
    prev = frames[0]

    out_img = None
    for i in range(1, len(frames)):
        curr = frames[i]
        p1, st, err = cv2.calcOpticalFlowPyrLK(prev, curr, p0, None, **lk_params)
        if p1 is None:
            break

        good_new = p1[st == 1]
        good_old = p0[st == 1]

        frame_bgr = cv2.cvtColor(curr, cv2.COLOR_GRAY2BGR)
        for (x1, y1), (x0, y0) in zip(good_new, good_old):
            x1, y1, x0, y0 = int(x1), int(y1), int(x0), int(y0)
            mask = cv2.line(mask, (x1, y1), (x0, y0), (0, 255, 0), 1)
            frame_bgr = cv2.circle(frame_bgr, (x1, y1), 2, (0, 0, 255), -1)

        out_img = cv2.add(frame_bgr, mask)
        prev = curr.copy()
        p0 = good_new.reshape(-1, 1, 2)

    # Show final frame with tracks
    if out_img is not None:
        plt.figure(figsize=(8, 6))
        plt.imshow(out_img[..., ::-1])  # BGR->RGB
        plt.title("Optical Flow Tracks (Synthetic Checkerboard)")
        plt.axis("off")
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    main()